In [25]:
import workflow.scripts.resources as res_module
import pandas as pd
import geopandas as gpd
import linkingtool.utility as utils

In [26]:
from linkingtool.hdf5_handler import DataHandler
import linkingtool.visuals as vis
store=f"data/store/resources_BC.h5"
bccm_store=f"/localhome/mei3/eliasinul/work/BC_Combined_Modelling/data/store/resources_BC.h5"
datahandler=DataHandler(store)

>> Store initialized with the given path: data/store/resources_BC.h5


In [27]:
datahandler.show_tree(bccm_store)

Structure of HDF5 file: /localhome/mei3/eliasinul/work/BC_Combined_Modelling/data/store/resources_BC.h5
[Group] boundary
[Group] cells
[Group] clusters
  └─ [Group] clusters/solar
  └─ [Group] clusters/wind
[Group] dissolved_indices
  └─ [Group] dissolved_indices/solar
  └─ [Group] dissolved_indices/wind
[Group] substations
[Group] timeseries
  └─ [Group] timeseries/clusters
  └─   └─ [Group] timeseries/clusters/solar
  └─   └─ [Group] timeseries/clusters/wind
  └─ [Group] timeseries/solar
  └─ [Group] timeseries/wind
[Group] units


In [28]:
sites=datahandler.from_store('clusters/solar')

FileNotFoundError: ``/local-scratch/localhome/mei3/eliasinul/work/BC_Combined_Modelling/models/Linking_tool/data/store/resources_BC.h5`` does not exist

In [ ]:
def select_top_sites(
    all_scored_sites_gdf:gpd.GeoDataFrame, 
    resource_max_capacity:float)-> gpd.GeoDataFrame:
    print(f">>> Selecting TOP Sites to for {resource_max_capacity} GW Capacity Investment in BC...")
    """
    Select the top sites based on potential capacity and a maximum resource capacity limit.

    Parameters:
    - sites_gdf: GeoDataFrame containing  cell and bucket information.
    - resource_max_capacity : Maximum allowable  capacity in GW.

    Returns:
    - selected_sites: GeoDataFrame with the selected top sites.
    """
    print(f"{'_'*50}")
    print(f"Selecting the Top Ranked Sites to invest in {resource_max_capacity} GW PV in BC")
    print(f"{'_'*50}\n")

    # Initialize variables
    selected_rows:list = []
    total_capacity:float = 0.0

    top_sites:gpd.GeoDataFrame = all_scored_sites_gdf.copy()

    if top_sites['potential_capacity'].iloc[0] < resource_max_capacity * 1000:
        # Iterate through the sorted GeoDataFrame
        for index, row in top_sites.iterrows():
            # Check if adding the current row's capacity exceeds resource capacity
            if total_capacity + row['potential_capacity'] <= resource_max_capacity * 1000:
                selected_rows.append(index)  # Add the row to the selection
                # Update the total capacity
                total_capacity += row['potential_capacity']
            # If adding the current row's capacity would exceed max resource capacity, stop the loop
            else:
                break

        # Create a new GeoDataFrame with the selected rows
        top_sites:gpd.GeoDataFrame = top_sites.loc[selected_rows]

        # Apply the additional logic
        mask = all_scored_sites_gdf['Site_ID'] > top_sites['Site_ID'].max()
        selected_additional_sites:gpd.GeoDataFrame = all_scored_sites_gdf[mask].head(1)
        
        remaining_capacity:float = resource_max_capacity * 1000 - top_sites['potential_capacity'].sum()

        if remaining_capacity > 0:
            
            # selected_additional_sites['capex'] = capex* remaining_capacity
            print(f"\n!! Note: The Last cluster originally had {round(selected_additional_sites['potential_capacity'].iloc[0] / 1000,2)} GW potential capacity."
                 f"To fit the maximum capacity investment of {resource_max_capacity} GW, it has been adjusted to {round(remaining_capacity / 1000,2)} GW\n")
            
            selected_additional_sites['potential_capacity'] = remaining_capacity
        # Concatenate the DataFrames
        top_sites = pd.concat([top_sites, selected_additional_sites])
    else:
        original_capacity = all_scored_sites_gdf['potential_capacity'].iloc[0]

        print(f"\n!! Note: The first cluster originally had {round(original_capacity / 1000,2)} GW potential capacity."
              f"To fit the maximum capacity investment of {resource_max_capacity} GW, it has been adjusted. \n")

        top_sites = top_sites.iloc[:1]  # Keep only the first row
        # Adjust the potential_capacity of the first row
        top_sites.at[top_sites.index[0], 'potential_capacity'] = resource_max_capacity * 1000

    return top_sites  # gdf

# Set Required Args to Activate Modules

In [40]:
# Iterate over provinces for both solar and wind resources
resource_types = ['wind'] # , 'solar'
provinces=['BC']  #'AB','SK','ON','NS'
for province_code in provinces:
    for resource_type in resource_types:
        required_args = {
            "config_file_path": 'config/config.yaml',
            "province_short_code": province_code,
            "resource_type": resource_type
        }
        
        # Create an instance of Resources and execute the module
        resources = res_module.Resources(**required_args)
        # resource_module.execute_module()
        

>> Store initialized with the given path: data/store/resources_BC.h5
>> Store initialized with the given path: data/store/resources_BC.h5
>> Store initialized with the given path: data/store/resources_BC.h5
>> Store initialized with the given path: data/store/resources_BC.h5
>> Store initialized with the given path: data/store/resources_BC.h5
>> Store initialized with the given path: data/store/resources_BC.h5


In [42]:
resources.find_grid_nodes()

2024-11-28 15:02:14,657 - INFO - >> Data pulled substations from [source checked: CODERS(https://sesit.dev/api/docs)]
2024-11-28 15:02:14,660 - INFO - substations data saved to:
 data/downloaded_data/CODERS/network/substations.pkl
2024-11-28 15:02:14,688 - INFO - >> Loading GADM boundaries (Sub-provincial | level =2) for British Columbia  from local file data/processed_data/regions/gadm41_Canada_L2_BC.geojson.
2024-11-28 15:02:14,801 - INFO - Building new cutout data/downloaded_data/cutout/BC_2023_2024.nc
2024-11-28 15:02:14,833 - INFO - Storing temporary files in /tmp/tmpn79hqhwd
2024-11-28 15:02:14,836 - INFO - Calculating and writing with module era5:
2024-11-28 15:02:14,839 - INFO - Requesting data for feature influx...
2024-11-28 15:02:14,841 - INFO - Requesting data for feature temperature...
2024-11-28 15:02:14,842 - INFO - Requesting data for feature wind...
2024-11-28 15:02:14,843 - INFO - Requesting data for feature runoff...
2024-11-28 15:02:14,845 - INFO - Requesting data f

KeyboardInterrupt: 

In [5]:
res_module.export_results('solar',
                          resources.get_clusters().clusters,
                          resources.get_cluster_timeseries())

2024-11-28 12:43:25,500 - INFO - >> Preparing spatial clusters for 2956 Cells


>> Calculating Score for each Cell...
>> Updated 'cells' saved to data/store/resources_BC.h5 with key 'cells'
>> Estimating optimal number of Clusters for each region based on the Score for each Cell ...
Zone Stikine - Optimal k for lcoe_wind based clustering: 4

>> Estimating optimal number of Clusters for each region based on the Score for each Cell ...
Zone Columbia-Shuswap - Optimal k for lcoe_wind based clustering: 4

>> Estimating optimal number of Clusters for each region based on the Score for each Cell ...
Zone Kitimat-Stikine - Optimal k for lcoe_wind based clustering: 5

>> Estimating optimal number of Clusters for each region based on the Score for each Cell ...
Zone CentralCoast - Optimal k for lcoe_wind based clustering: 4

>> Estimating optimal number of Clusters for each region based on the Score for each Cell ...
Zone CentralKootenay - Optimal k for lcoe_wind based clustering: 4

>> Estimating optimal number of Clusters for each region based on the Score for each Cell 

2024-11-28 12:43:29,280 - INFO -  Preparing Clusters...
2024-11-28 12:43:29,280 - INFO -  Creating cluster for Stikine 1/28
2024-11-28 12:43:29,299 - INFO -  Creating cluster for Columbia-Shuswap 2/28
2024-11-28 12:43:29,314 - INFO -  Creating cluster for Kitimat-Stikine 3/28
2024-11-28 12:43:29,366 - INFO -  Creating cluster for CentralCoast 4/28
2024-11-28 12:43:29,416 - INFO -  Creating cluster for CentralKootenay 5/28
2024-11-28 12:43:29,436 - INFO -  Creating cluster for Comox-Strathcona 6/28
2024-11-28 12:43:29,480 - INFO -  Creating cluster for Cariboo 7/28
2024-11-28 12:43:29,502 - INFO -  Creating cluster for MountWaddington 8/28
2024-11-28 12:43:29,552 - INFO -  Creating cluster for NorthOkanagan 9/28
2024-11-28 12:43:29,568 - INFO -  Creating cluster for PowellRiver 10/28
2024-11-28 12:43:29,585 - INFO -  Creating cluster for SunshineCoast 11/28
2024-11-28 12:43:29,603 - INFO -  Creating cluster for Squamish-Lillooet 12/28
2024-11-28 12:43:29,619 - INFO -  Creating cluster f

>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_BC.h5 with key 'clusters/wind'
>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_BC.h5 with key 'dissolved_indices/wind'
>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_BC.h5 with key 'timeseries/clusters/wind'
solar clusters exported to :results
